# Spherical harmonics vs graph Fourier modes

[Nathanaël Perraudin](http://perraudin.info), [Michaël Defferrard](http://deff.ch), Tomasz Kacprzak

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
import os
from time import time

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import healpy as hp
import pygsp as pg
import itertools

from scnn import utils

In [ ]:
plt.rcParams['figure.figsize'] = (17, 5)  # (9, 4) for matplotlib notebook

nside = 16
cm = plt.cm.RdBu_r
cm.set_under('w')

## 1 Spherical harmonics

* Spherical harmonics are indexed by degree (or angular frequency) $\ell$ and order $m \in [0, \ell]$.
* $l=0$ is the constant, $l=1$ is the two monopoles, $l=2$ is the three dipoles, etc.
* Coefficients are commonly defined as $a_{\ell m}$.
* A spherical harmonic takes value $Y_\ell^m(\theta, \phi)$ in angular direction $(\theta, \phi)$.
* In HEALPix, as we only deal with maps of real numbers, the representation is complex and the relation $a_{\ell, m}^* = a_{\ell, -m}$ holds. As such, to generate all $2\ell + 1$ orders, one needs to e.g. transform $a_{\ell m} = 1 + 1j$ and $a_{\ell m} = 1 - 1j$.

In [ ]:
def plot_spherical_harmonic(nside, l, m):

    lmax = l
    idx = hp.sphtfunc.Alm.getidx(lmax, l, m)
    size = hp.sphtfunc.Alm.getsize(lmax, mmax=lmax)
    print('{} spherical harmonics for l in [0, {}]'.format(size, lmax))
    print('l={}, m={} is at index {}'.format(l, m, idx))

    alm = np.zeros(size, dtype=np.complex128)
    alm[idx] = 1

    map = hp.sphtfunc.alm2map(alm, nside, lmax, verbose=False)
    hp.mollview(map, title="Spherical harmonic of degree l={} and order m={}".format(l,m), cmap=cm)
    #hp.cartview(map, title=f"Spherical harmonic l={l}, m={m}")
    with utils.HiddenPrints():
        hp.graticule();
plot_spherical_harmonic(nside, l=2, m=0)

Compute the spherical harmonics up to $\ell_{max}$ and plot them.

In [ ]:
harmonics = utils.compute_spherical_harmonics(nside, lmax=18)
harmonics.shape

In [ ]:
def plot_harmonics(harmonics, title=''):
    n_harmonics = harmonics.shape[1]
    l, m = 0, 0
    for idx in range(n_harmonics):
        hp.mollview(harmonics[:, idx], 
                    title='{}: l={}, m={}'.format(title, l, m),
                    nest=True,
                    sub=(np.sqrt(n_harmonics), np.sqrt(n_harmonics), idx+1),
                    max=np.max(np.abs(harmonics)),
                    min=-np.max(np.abs(harmonics)),
                    cbar=False,
                    cmap=cm)
        
        m += 1
        if m > l:
            l += 1
            m = -l
    with utils.HiddenPrints():
        hp.graticule();
plot_harmonics(harmonics[:, :16], 'Spherical harmonic')
plt.savefig('figures/spherical_harmonics.png')

## 2 Graph Fourier modes

The graph Fourier modes are the eigenvectors of the graph Laplacian.
$$L = U \Lambda U^T$$

In [ ]:
graph = utils.healpix_graph(nside, lap_type='normalized', nest=True, dtype=np.float64)
graph.compute_fourier_basis()

The weighted adjacency matrix is very sparse. Distance between pixels is not constant.

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(graph.W.toarray())
axes[1].hist(graph.W[graph.W>0].T,30);

Fourier modes of the graph.

In [ ]:
plot_harmonics(graph.U[:, :16], 'Eigenvector')
plt.savefig('figures/graph_eigenvectors.png')

The eigenvalues are clearly organized in groups, which corresponds to angular frequencies $\ell$ of the spherical harmonics.

In [ ]:
plt.plot(graph.e[:50], '.-')
idx = 0
for l in range(7):
    plt.text(idx, graph.e[idx] + 0.005, 'l = {}'.format(l))
    idx += 2*l + 1
plt.savefig('figures/graph_eigenvalues.png')

## 3 Spherical harmonics on the graph

Todo:
* Smoothness of rotated spherical harmonics should be constant.

In [ ]:
# Combinatorial Laplacian for the spherical harmonics.
graph = utils.healpix_graph(nside, lap_type='combinatorial', nest=True, dtype=np.float64)
graph.compute_fourier_basis()

Let us try to re-order the spherical harmonic with respect of the graph frequencies

In [ ]:
ind = np.argsort(np.diag(harmonics.T @ graph.L @ harmonics))
harmonics_sort = harmonics[:,ind]

Spherical harmonics are not exactly orthogonal on the graph.

In [ ]:
C_euclidean = harmonics_sort.T @ harmonics_sort
C_graph = (harmonics_sort.T @ graph.L @ harmonics_sort) #@ np.diag(1/(graph.e[:harmonics_sort.shape[1]] +0.0001))

fig, axes = plt.subplots(1, 2)
axes[0].imshow(C_euclidean)
axes[1].imshow(C_graph)

fig, axes = plt.subplots(1, 2)
axes[0].plot(np.diag(C_euclidean))
axes[1].plot(np.diag(C_graph), label='diag(F^T L F)')
axes[1].plot(graph.e[:harmonics_sort.shape[1]],label='graph eigenvalues')
axes[1].legend()

## 4 Correspondance of the subspaces

* Are the subspaces equivalent?
* Is the projection on the subspaces similar?

In [ ]:
lmax = 10
n_harmonics = np.sum(np.arange(1, 2*lmax+2, 2))
print('{} harmonics for lmax = {}'.format(n_harmonics, lmax))

In [ ]:
C = harmonics[:, :n_harmonics].T @ harmonics[:, :n_harmonics]
print(np.linalg.norm(C, ord='fro'))

C = graph.U[:, :n_harmonics].T @ graph.U[:, :n_harmonics]
print(np.linalg.norm(C, ord='fro'))

C = harmonics[:, :n_harmonics].T @ graph.U[:, :n_harmonics]
print(np.linalg.norm(C, ord='fro'))

plt.imshow(np.abs(C), cmap=plt.cm.gist_heat_r)
plt.colorbar()
plt.savefig('figures/subspace_harmonics_eigenvectors.png')

In [ ]:
# This corresponds to a low pass filtering

n_eigenvectors = 16
n_harmonics = 16
n_signals = 100
n_pixels = harmonics.shape[0]

signals = np.random.uniform(size=(n_pixels, n_signals))
eigenvectors = graph.U[:, :n_eigenvectors]
harmonics_ = harmonics[:, :n_harmonics]

signals_sphere = harmonics_ @ harmonics_.T @ signals
signals_graph = eigenvectors @ eigenvectors.T @ signals

hp.mollview(signals[:, 0], nest=True)
hp.mollview(signals_sphere[:, 0], nest=True)
hp.mollview(signals_graph[:, 0], nest=True)

print(np.linalg.norm(signals_graph - signals_sphere, ord='fro'))

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(graph.U[:,1], graph.U[:,2], graph.U[:,3], c=graph.d)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(harmonics[:,1], harmonics[:,2], harmonics[:,3], c=graph.d)

## 5 Accuracy of the filtering

We don't care if the Fourier modes are not exactly the spherical harmonics, but we want the Laplacian operator to be close. Or, at least, the filtering operations to be close.

* Does applying L change the spectrum (from spherical harmonics) in a meaningful way?

Functions:
* `healpy.sphtfunc.map2alm` => arbitrary smoothing
* `healpy.sphtfunc.` => Gaussian smoothing

In [ ]:
nside = 256

map, _, _ = hp.read_map('data/COM_CMB_IQU-smica_1024_R2.02_full.fits', field=(0, 1, 3), nest=True)
map = hp.ud_grade(map, nside_out=nside, order_in='NESTED')
map = hp.reorder(map, inp='NESTED', out='RING')
# map = np.random.normal(size=hp.nside2npix(nside))

lmax = 3 * nside - 1

print("We'll compute {} frequencies".format(lmax))
hp.mollview(map)

In [ ]:
alm = hp.sphtfunc.map2alm(map)
plt.plot(np.abs(alm))

l, m = hp.Alm.getlm(lmax=lmax)
print(list(zip(l, m))[:100])

In [ ]:
cl = hp.sphtfunc.anafast(map)
plt.plot(cl)

In [ ]:
graph = utils.healpix_graph(nside, lap_type='normalized', nest=True, dtype=np.float64)
map = graph.L @ map

In [ ]:
alm = hp.sphtfunc.map2alm(map)
plt.plot(np.abs(alm))

In [ ]:
cl = hp.sphtfunc.anafast(map)
plt.plot(cl)

## 6 Spherical power spectrum

The angular power spectrum is given by
$$ \hat{C}_\ell = \frac{1}{2\ell + 1} \sum_m |\hat{a}_{\ell m}|^2 $$
As such, $\hat{C}_\ell$ is the expected variance of the $\hat{a}_{\ell m}$ at order $\ell$.
It also implies the standard result that the total power at the angular wavenumber $\ell$ is $(2\ell + 1) \hat{C}_\ell$ because there are $2\ell + 1$ modes for each $\ell$.

Todo:
* speed
* accuracy

In [ ]:
psd = hp.sphtfunc.anafast(map, lmax=lmax)
plt.semilogy(psd);

## 7 Filtering speed

Todo:
* filtering with tensorflow / numpy / pytorch
* Similarly for a part of the sphere
* speed and accuracy w.r.t. polynomial order

Functions:
* `healpy.sphtfunc.almxfl` => arbitrary smoothing
* `healpy.sphtfunc.smoothing` => Gaussian smoothing

In [ ]:
nside = 256

map, _, _ = hp.read_map('data/COM_CMB_IQU-smica_1024_R2.02_full.fits', field=(0, 1, 3), nest=True)
map = hp.ud_grade(map, nside_out=nside, order_in='NESTED')
map = hp.reorder(map, inp='NESTED', out='RING')

hp.mollview(map)

Smoothing by filtering with spherical harmonics, i.e. do an element-wise multilication in the spectral domain after a spherical transform.

In [ ]:
smooth = hp.sphtfunc.smoothing(map, sigma=0.01, verbose=False)
hp.mollview(smooth)

Smoothing by filtering on graphs.

In [ ]:
graph = utils.healpix_graph(nside, lap_type='normalized', nest=True, dtype=np.float64)
graph.estimate_lmax()

map = hp.reorder(map, inp='RING', out='NESTED')

filter = pg.filters.Heat(graph, tau=30)
filter = filter.approximate('Chebyshev', order=10)
filter.plot()

smooth = filter.filter(map)
hp.mollview(smooth, nest=True)

Compare the speed of both approaches.

We limit OpenMP (used by HEALPix) to use a single core as graph convolutions are not parallelized yet. While the mutliplication of sparse matrices with dense vectors can be parallelized, it is not implemented by scipy. It's hopefully implemented in tensorflow or pytorch.

In [ ]:
nsides = [64, 128, 256, 512, 1024, 2048]  # Max 1024 for the real Planck map.

# Polynomial orders for graph filtering.
orders = [5, 10, 20]

# Number of OpenMP threads for spherical harmonics.
os.environ['OMP_NUM_THREADS'] = '1'

times = np.zeros((len(nsides), len(orders)+1))

for i, nside in enumerate(nsides):

    # map = hp.ud_grade(map_cmb, nside_out=nside, order_in='NESTED')
    map = np.random.normal(size=hp.nside2npix(nside))

    # Filtering on the graph. Need the nested ordering.
    graph = utils.healpix_graph(nside, lap_type='normalized', nest=True, dtype=np.float64)
    graph.estimate_lmax()
    for j, order in enumerate(orders):
        filter = pg.filters.Heat(graph, tau=30).approximate('Chebyshev', order=order)
        t = time()
        smooth = filter.filter(map)
        times[i, j] = time() - t

    # Filtering with the spherical harmonics. Need the ring ordering.
    map = hp.reorder(map, inp='NESTED', out='RING')
    t = time()
    hp.sphtfunc.smoothing(map, sigma=0.01, verbose=False)
    times[i, -1] = time() - t

# Reset OpenMP to use all available cores.
del os.environ['OMP_NUM_THREADS']

In [ ]:
# Save results for the plot in the paper.
np.savez('results/filtering_speed.npz', nsides=nsides, orders=orders, times=times)

In [ ]:
fig, ax = plt.subplots()
npix = [hp.nside2npix(nside) for nside in nsides]
ax.loglog(npix, times[:, :-1], '.-')
ax.loglog(npix, times[:, -1], '.-k', linewidth=3, markersize=10)
labels = ['Filtering with graphs, polynomial order {}'.format(order) for order in orders]
labels += ['Filtering with spherical harmonics']
ax.legend(labels)
for i in range(len(nsides)):
    ax.text(npix[i], times[i, -1] * 1.8, 'Nside = {}'.format(nsides[i]), horizontalalignment='center')
ax.set_ylim(0.8 * times.min(), 3 * times.max())
ax.set_xlabel('Number of pixels')
ax.set_ylabel('Processing time [s]')
ax.set_title('Single core performance')
#ax.ticklabel_format(style='sci', axis='x')
fig.savefig('figures/filtering_speed.png')

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(orders, times[:, :-1].T, '.-')
labels = ['Filtering with graphs, Nside = {}'.format(nside) for nside in nsides]
ax.legend(labels)
ax.set_xlabel('Polynomial order')
ax.set_xticks(orders)
ax.set_ylabel('Processing time [s]');

In [ ]:
fig, ax = plt.subplots()
npix = [hp.nside2npix(nside) for nside in nsides]
ax.plot(nsides, times, '.-')
ax.legend(labels)
ax.set_xlabel('Nsides')
ax.set_xticks(nsides)
ax.set_ylabel('Processing time [s]');

## 8 Convergence when number of pixels goes to infinity

As a graph representation of the sphere is not dependant on the sampling, we can compute spherical harmonics from random samplings of the sphere. As the sampling increases, the eigenvectors approach the spherical harmonics.

Do the eigenvalues converge to the orders $\ell$, i.e. do the stairs become flatter, when increasing $N_{pix}$? No, they mostly stay the same.

In [ ]:
n_points = [100, 500, 1000]
#fig, axes = plt.subplots(1, len(n_points))

for i, n_points in enumerate(n_points):
    graph = pg.graphs.Sphere(nb_pts=n_points)
    graph.compute_fourier_basis(n_eigenvectors=n_harmonics)
    graph.plot_signal(graph.U[:, 1], edges=False)#, ax=axes[0])
    plt.axis('off')

## 9 Part of sphere

Tweak the part graph such that the eigenvectors computed on parts of the sphere are as if we would have restricted the global eigenvectors to a part of sphere.

Todo: Mais tu peux essayer sur une grille normale. Le plus simple pour vérifier si c'est possible, c'est de vérifier s'il est nécessaire d'avoir une matrice circulante pour avoir des exponentielles complexes comme vecteurs propres.